<h1 align=center>Redes Adversariais Geradoras para Sintetizar Novos Dados</h1>

<p align=center><img src=https://advancedinstitute.ai/wp-content/uploads/2020/08/real_fake.png width=500></p>

Anteriomente, focamos em redes neurais recorrentes (RNN) para modelagem de sequências. Aqui, exploraremos as redes generativas de adversários (GANs) e veremos sua aplicação na síntese de novas amostras de dados. As GANs são consideradas o avanço mais importante no aprendizado profundo, permitindo que os computadores gerem novos dados (como novas imagens).

Aqui, abordaremos os seguintes tópicos:
* Apresentando modelos generativos para sintetizar novos dados
* Autoencoders, autoencoders variacionais (VAEs) e seu relacionamento com GANs
* Entendendo os blocos de construção das GANs
* Implementação de um modelo GAN simples para gerar dígitos manuscritos
* Noções básicas sobre convolução transposta e normalização de lote (BatchNorm ou BN)
* Melhorar GANs: GANs convolucionais profundos e GANs usando a distância de Wasserstein

## Apresentando redes adversárias generativas
Vamos primeiro olhar para os fundamentos dos modelos GAN. O objetivo geral de uma GAN é <u>sintetizar novos dados que tenham a mesma distribuição que seu conjunto de dados de treinamento</u>. Portanto, as GANs, em sua forma original, são consideradas na categoria de aprendizado não supervisionado de tarefas de aprendizado de máquina, pois não são necessários dados rotulados. Vale a pena notar, no entanto, que as extensões feitas ao GAN original podem estar em tarefas semi-supervisionadas e supervisionadas.

O conceito geral de GAN foi proposto pela primeira vez em 2014 por Ian Goodfellow e seus colegas como um método para sintetizar novas imagens usando redes neurais profundas (NNs). Embora a arquitetura GAN inicial proposta fosse baseada em camadas totalmente conectadas, semelhantes às arquiteturas *perceptron* multicamadas, e treinadas para gerar dígitos manuscritos de baixa resolução do tipo MNIST, ela serviu mais como uma prova de conceito para demonstrar a viabilidade dessa nova abordagem.

No entanto, desde sua introdução, os autores originais, assim como muitos outros pesquisadores, propuseram inúmeras melhorias e diversas aplicações em diferentes campos da engenharia e da ciência; por exemplo, em visão computacional, GANs são usadas ​​para tradução de imagem para imagem (aprender a mapear uma imagem de entrada para uma imagem de saída), super-resolução de imagem (criar uma imagem de alta resolução a partir de uma versão de baixa resolução), pintura interna de imagem (aprender a reconstruir as partes que faltam de uma imagem) e muitas outras aplicações. Por exemplo, os avanços recentes na pesquisa de GAN levaram a modelos capazes de gerar novas imagens de rosto de alta resolução. Exemplos dessas imagens de alta resolução podem ser encontrados em https://www.thispersondoesnotexist.com/, que mostra imagens de rosto sintéticas geradas por uma GAN.

### Começando com os *autoencoders*
Antes de discutirmos como as GANs funcionam, começaremos com os *autoencoders*, que podem compactar e descompactar dados de treinamento. Embora os codificadores automáticos padrão não possam gerar novos dados, entender sua função ajudará você a navegar pelas GANs na próxima seção.

Autoencoders são compostos por duas redes concatenadas: uma rede codificadora e uma rede decodificadora. A rede codificadora recebe um vetor de recurso de entrada *d*-dimensional associado ao exemplo **x** (ou seja, ***x*** $\small \in R^p$) e o codifica em um vetor *p*-dimensional, `z` (ou seja, ***z*** $\small \in R^p$). Em outras palavras, o papel do codificador é aprender a modelar a função ***z*** = $\small f(x)$ . O vetor codificado, `z`, também é chamado de vetor latente ou representação de característica latente. Normalmente, a dimensionalidade do vetor latente é menor que a dos exemplos de entrada; em outras palavras, $\small p < d$. Assim, podemos dizer que o codificador atua como uma função de compressão de dados. Então, o decodificador descomprime $\small \hat{x}$ do vetor latente de menor dimensão, `z`, onde podemos pensar no decodificador como uma função, $\small \hat{x} = g(z)$. Uma arquitetura simples de *autoencoder* é mostrada na figura a seguir, onde as partes do codificador e do decodificador consistem em apenas uma camada totalmente conectada cada:

![](imagens\autoencoder.PNG)

> ##### A conexão entre autoencoders e redução de dimensionalidade
> Certamente você já aprendeu sobre técnicas de redução de dimensionalidade, como análise de componentes principais (PCA) e análise discriminante linear (LDA). *Autoencoders* também podem ser usados como uma **técnica de redução de dimensionalidade**. De fato, quando não há não linearidade em nenhuma das duas sub-redes (codificador e decodificador), a abordagem do autoencoder é quase idêntica ao PCA.
>
> Nesse caso, se assumirmos que os pesos de um codificador de camada única (sem camada oculta e sem função de ativação não linear) são denotados pela matriz **U**, então os modelos do codificador **z** = $\small U^Tx$. Da mesma forma, um decodificador linear de camada única modela $\small \hat{x} = Uz$. Juntando esses dois componentes, temos $\small \hat{x} = UU^Tx$. Isso é exatamente o que o PCA faz, com a exceção de que o PCA tem uma restrição ortonormal adicional: $\small UU^T = I_{n \times n}$. 

Embora a figura anterior represente um *autoencoder* sem camadas ocultas dentro do codificador e do decodificador, podemos, é claro, adicionar várias camadas ocultas com não linearidades (como em uma NN multicamada) para construir um *autoencoder* profundo que pode aprender funções de reconstrução e compactação de dados mais eficazes. Além disso, observe que o *autoencoder* mencionado nesta seção usa camadas totalmente conectadas. Quando trabalhamos com imagens, no entanto, podemos substituir as camadas totalmente conectadas por camadas convolucionais.

> ##### Outros tipos de autoencoders baseados no tamanho do espaço latente
> Como mencionado anteriormente, a dimensionalidade do espaço latente de um *autoencoder* é tipicamente menor que a dimensionalidade das entradas ($\small p < d$), o que torna os autoencoders adequados para redução de dimensionalidade. Por esse motivo, o vetor latente também é frequentemente chamado de "gargalo", e essa configuração específica de um *autoencoder* também é chamada de `undercomplete` (subcompleto). No entanto, existe uma categoria diferente de *autoencoders*, chamada `overcomplete`, onde a dimensionalidade do vetor latente, `z`, é, de fato, maior que a dimensionalidade dos exemplos de entrada ($\small p > d$).

Ao treinar um *autoencoder* `overcomplete` (supercompleto), há uma solução trivial em que o codificador e o decodificador podem simplesmente aprender a copiar (memorizar) os recursos de entrada para sua camada de saída. Obviamente, esta solução não é muito útil. No entanto, com algumas modificações no procedimento de treinamento, os *autoencoders* supercompletos podem ser usados **​​para redução de ruído**. Nesse caso, durante o treinamento, o ruído aleatório, $\epsilon$, é adicionado aos exemplos de entrada e a rede aprende a reconstruir o exemplo, `x`, do sinal ruidoso, $\small x + \epsilon$. Então, no momento da avaliação, fornecemos os novos exemplos que são naturalmente ruidosos (ou seja, o ruído já está presente de modo que nenhum ruído artificial adicional, $\epsilon$, é adicionado) para remover o ruído existente desses exemplos. Essa arquitetura de *autoencoder* e o método de treinamento específicos são chamados de *autoencoder* de redução de ruído.

### Modelos generativos para sintetizar novos dados